# Loop InductionVariables

在编译原理课程项目中，找归纳变量的代码，需要提供循环不变量的信息。

下面先找循环，再找归纳变量。

## 需要的库

- 正则表达式 re
- 序列化 json

In [1]:
import re
import json
import copy

## 读入json

In [14]:
filename = "../examples/quicksort_src_blk.json"
vm_blk = {}
with open(filename, "r") as fp:
    vm_blk = json.load(fp)
blocks = vm_blk["blocks"]

cond_f = {"==": lambda x, y: x == y, "!=": lambda x, y: x != y, ">=": lambda x, y: x >=
          y, "<=": lambda x, y: x <= y, ">": lambda x, y: x > y, "<": lambda x, y: x < y}
arith_f = {"+": lambda x, y: x+y, "-": lambda x, y: x-y, "*": lambda x,
           y: x*y, "/": lambda x, y: x//y, "%": lambda x, y: x % y}
eserved_word = {"HALT", "=", "+", "-", "*", "/", "%", "?",
                ":", "!:", ">", "<", ">=", "<=", "==", "!=", "[", "]"}
reg = re.compile('^[0-9]+$')

In [15]:
blocks

{'0': {'line_num': [0, 3],
  'next': [1, None],
  'code': ['SL [ 0 ] = 0', 'N = N - 1', 'SR [ 0 ] = N', 'TOP = 0'],
  'defd': ['SL', 'SR', 'TOP'],
  'used': ['N'],
  'in': ['A', 'N'],
  'out': ['SR', 'SL', 'A', 'TOP']},
 '1': {'line_num': [4, 4],
  'next': [2, 9],
  'code': ['? TOP < 0 : 47'],
  'defd': [],
  'used': ['TOP'],
  'in': ['SL', 'A', 'SR', 'TOP'],
  'out': ['A', 'SL', 'SR', 'TOP']},
 '2': {'line_num': [5, 8],
  'next': [3, 1],
  'code': ['M = SL [ TOP ]',
   'N = SR [ TOP ]',
   'TOP = TOP - 1',
   '? N <= M : 4'],
  'defd': ['N', 'M'],
  'used': ['SL', 'SR', 'TOP'],
  'in': ['SL', 'A', 'SR', 'TOP'],
  'out': ['M', 'SR', 'N', 'SL', 'A', 'TOP']},
 '3': {'line_num': [9, 12],
  'next': [4, None],
  'code': ['I = M - 1', 'J = N', 'T_1 = N', 'V = A [ T_1 ]'],
  'defd': ['J', 'V', 'I', 'T_1'],
  'used': ['A', 'N', 'M'],
  'in': ['N', 'A', 'M', 'TOP'],
  'out': ['M', 'J', 'I', 'V', 'N', 'A', 'TOP']},
 '4': {'line_num': [13, 16],
  'next': [5, 4],
  'code': ['I = I + 1', 'T_2 = I',

# 找循环

In [16]:
# 计算基本块的前驱
for n,b in blocks.items():
    blocks[n]["pre"] = set()
for n,b in blocks.items():
    for i in b["next"]:
        if i!=None:
            blocks[str(i)]["pre"] = blocks[str(i)]["pre"] | {n}
# 构建支配关系
for n,b in blocks.items():
    blocks[n]["dom"] = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
blocks[str(0)]["dom"] = {'0'}
flag = True
i = 0
while flag:
    flag = False
    i+=1
    for n,b in blocks.items():
        if n!='0':
            domset_ori = copy.deepcopy(blocks[n]["dom"])
            domset = copy.deepcopy(blocks[n]["dom"])
            preset = set([str(i) for i in range(vm_blk["summary"]["total_blocks"])])
            for j in blocks[n]["pre"]:
                preset = preset & blocks[j]["dom"]
            domset = {n} | preset
            if (domset!=domset_ori):
                flag = True
            blocks[n]["dom"] = domset
for n,b in blocks.items():
    if len(b["pre"])==0 and n!='0':
        blocks[n]["dom"] = set()
# 寻找回边
back_edge = []
for n,b in blocks.items():
    for i in b["dom"]:
        if int(i) in b["next"]:
            back_edge.append((n,i))
back_edge = list(set(back_edge))
print(back_edge)
# 寻找回边对应的自然循环
loops = []
for le in back_edge:
    stack = [le[0]]
    loop = {le[0],le[1]}
    if le[0]==le[1]:
        continue
    while len(stack)>0:
        m=stack[-1]
        stack.pop()
        for p in blocks[m]["pre"]:
            if p not in loop:
                loop = loop | {p}
                stack.append(p)
    loop = [i for i in list(loop) if len(blocks[i]["pre"])>0]
    # print(le,sorted(loop))
    loops.append((sorted(loop),le))
# 找只有一个基本块的循环
for n,b in blocks.items():
    if int(n) in b["next"]:
        loops.append(([n],(n,n)))
loops = sorted(loops,key = lambda x:len(x[0]))
# loops = list(set(loops))
loops_res = []
for l in loops:
    if l not in loops_res:
        loops_res.append(l)
print(loops_res)
for n,b in blocks.items():
    blocks[n]["dom"] = list(blocks[n]["dom"])
    blocks[n]["pre"] = list(blocks[n]["pre"])
vm_blk["blocks"] = blocks
vm_blk["loops"] = loops_res

[('2', '1'), ('8', '1'), ('4', '4'), ('5', '5'), ('7', '4')]
[(['4'], ('4', '4')), (['5'], ('5', '5')), (['1', '2'], ('2', '1')), (['4', '5', '6', '7'], ('7', '4')), (['1', '2', '3', '4', '5', '6', '7', '8'], ('8', '1'))]


# 读入循环不变量信息

In [18]:
invariant_var = [[] for i in loops_res]
invariant_var[0] = ["1"]
invariant_var[1] = ["1"]
invariant_var[2] = ["1"]
invariant_var

[['1'], ['1'], ['1'], [], []]

# 找基本归纳变量

In [24]:
basic_induc_var = [[] for i in loops_res]
for n,lp in enumerate(loops_res):
    for i in lp[0]:
        bcode = blocks[i]["code"]
        print(bcode)
        for line in bcode:
            symbols = line.split(" ")
            if symbols[1]=="=" and symbols[0]==symbols[2] and symbols[3] in ['+','-'] and symbols[4] in invariant_var[n]:
                basic_induc_var[n].append(symbols[0])
                print(line)

['I = I + 1', 'T_2 = I', 'T_3 = A [ T_2 ]', '? V > T_3 : 13']
I = I + 1
['J = J - 1', 'T_4 = J', 'T_5 = A [ T_4 ]', '? T_5 > V : 17']
J = J - 1
['? TOP < 0 : 47']
['M = SL [ TOP ]', 'N = SR [ TOP ]', 'TOP = TOP - 1', '? N <= M : 4']
TOP = TOP - 1
['I = I + 1', 'T_2 = I', 'T_3 = A [ T_2 ]', '? V > T_3 : 13']
['J = J - 1', 'T_4 = J', 'T_5 = A [ T_4 ]', '? T_5 > V : 17']
['? I >= J : 31']
['T_6 = I', 'X = A [ T_6 ]', 'T_7 = I', 'T_8 = J', 'T_9 = A [ T_8 ]', 'A [ T_7 ] = T_9', 'T_10 = J', 'A [ T_10 ] = X', '!: 13']
['? TOP < 0 : 47']
['M = SL [ TOP ]', 'N = SR [ TOP ]', 'TOP = TOP - 1', '? N <= M : 4']
['I = M - 1', 'J = N', 'T_1 = N', 'V = A [ T_1 ]']
['I = I + 1', 'T_2 = I', 'T_3 = A [ T_2 ]', '? V > T_3 : 13']
['J = J - 1', 'T_4 = J', 'T_5 = A [ T_4 ]', '? T_5 > V : 17']
['? I >= J : 31']
['T_6 = I', 'X = A [ T_6 ]', 'T_7 = I', 'T_8 = J', 'T_9 = A [ T_8 ]', 'A [ T_7 ] = T_9', 'T_10 = J', 'A [ T_10 ] = X', '!: 13']
['T_11 = I', 'X = A [ T_11 ]', 'T_12 = I', 'T_13 = N', 'T_14 = A [ T_13 ]

In [25]:
for n,i in enumerate(basic_induc_var):
    print("{} : {}".format(loops_res[n],i))

(['4'], ('4', '4')) : ['I']
(['5'], ('5', '5')) : ['J']
(['1', '2'], ('2', '1')) : ['TOP']
(['4', '5', '6', '7'], ('7', '4')) : []
(['1', '2', '3', '4', '5', '6', '7', '8'], ('8', '1')) : []


# 寻找同族归纳变量

In [26]:
# 待续

[{'I': []}, {'J': []}, {'TOP': []}, {}, {}]